In [1]:
import json  # Handles JSON data
import re  # Provides regular expression matching
import google.auth  # Handles Google authentication
from googleapiclient.discovery import build  # Google API client for service discovery
from googleapiclient.errors import HttpError  # Handles errors from Google API calls
from google.colab import auth  # Google Colab authentication module
from markdown import markdown  # Converts Markdown text to HTML
import unittest  # Python unit testing framework

In [2]:
def authenticate_google_docs():
    """Authenticates with Google Docs API and returns a service object."""
    try:
        auth.authenticate_user()
        creds, _ = google.auth.default()
        service = build('docs', 'v1', credentials=creds)
        print("Successfully authenticated with Google Docs API")
        return service
    except Exception as e:
        print(f"Authentication failed: {e}")
        raise

def create_google_doc(service, title):
    """Creates a new Google Doc with the given title and returns its document ID."""
    try:
        doc = service.documents().create(body={'title': title}).execute()
        doc_id = doc['documentId']
        print(f"Created document: {title}")
        print(f"Document ID: {doc_id}")
        print(f"Document URL: https://docs.google.com/document/d/{doc_id}")
        return doc_id
    except HttpError as e:
        print(f"Failed to create document: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error creating document: {e}")
        raise

def update_google_doc(service, doc_id, requests):
    """Updates the specified Google Doc with the given list of requests."""
    try:
        if not requests:
            print("No formatting requests to apply")
            return

        # Process requests in smaller batches to avoid API limits
        batch_size = 100
        for i in range(0, len(requests), batch_size):
            batch = requests[i:i + batch_size]
            service.documents().batchUpdate(
                documentId=doc_id,
                body={'requests': batch}
            ).execute()
            print(f"Applied batch {i//batch_size + 1} of {(len(requests)-1)//batch_size + 1}")

        print(f'Document updated successfully: https://docs.google.com/document/d/{doc_id}')

    except HttpError as e:
        print(f'Google API error occurred: {e}')
        if hasattr(e, 'resp') and hasattr(e.resp, 'status'):
            if e.resp.status == 403:
                print("Permission denied. Check document sharing settings.")
            elif e.resp.status == 404:
                print("Document not found. Check document ID.")
            elif e.resp.status == 429:
                print("Rate limit exceeded. Try again later.")
        raise
    except Exception as e:
        print(f'Unexpected error updating document: {e}')
        raise

In [3]:
def parse_markdown(md_text):
    """Converts markdown text into Google Docs API requests with proper formatting."""
    requests = []
    index = 1  # Google Docs uses 1-based indexing
    lines = md_text.split("\n")

    for line in lines:
        if not line.strip():  # Skip empty lines
            continue

        original_line = line
        actual_text = line.strip()
        style = None
        apply_footer_style = False
        is_bullet = False
        is_checkbox = False
        indent_level = 0

        # Calculate indentation level for bullets
        if line.lstrip().startswith(('- ', '* ')):
            indent_level = (len(line) - len(line.lstrip())) // 2
            is_bullet = True
            actual_text = line.lstrip()[2:].strip()

        # Handle checkboxes
        elif line.strip().startswith('- [ ]'):
            is_checkbox = True
            actual_text = line.strip()[5:].strip()
        elif line.strip().startswith('- [x]'):
            is_checkbox = True
            actual_text = "✓ " + line.strip()[5:].strip()

        # Handle markdown headings
        elif line.startswith("# "):
            actual_text = line[2:].strip()
            style = "HEADING_1"
        elif line.startswith("## "):
            actual_text = line[3:].strip()
            style = "HEADING_2"
        elif line.startswith("### "):
            actual_text = line[4:].strip()
            style = "HEADING_3"

        # Handle footer styling
        elif (line.lower().startswith("meeting recorded by") or
              line.lower().startswith("duration") or
              line.strip() == "---"):
            apply_footer_style = True
            if line.strip() == "---":
                actual_text = "─" * 50  # Horizontal rule

        # Insert text with proper formatting
        text_to_insert = actual_text + "\n"

        # Add bullet point symbol for bullets
        if is_bullet:
            indent_spaces = "  " * indent_level
            text_to_insert = f"{indent_spaces}• {actual_text}\n"
        elif is_checkbox:
            text_to_insert = f"☐ {actual_text}\n"

        requests.append({
            "insertText": {
                "location": {"index": index},
                "text": text_to_insert
            }
        })

        text_length = len(text_to_insert)

        # Apply heading styles
        if style:
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + text_length - 1},
                    "paragraphStyle": {"namedStyleType": style},
                    "fields": "namedStyleType"
                }
            })

        # Apply footer styling
        if apply_footer_style:
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": index, "endIndex": index + text_length - 1},
                    "textStyle": {
                        "italic": True,
                        "fontSize": {"magnitude": 10, "unit": "PT"}
                    },
                    "fields": "italic,fontSize"
                }
            })

        # Apply bullet point formatting
        if is_bullet and indent_level > 0:
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + text_length - 1},
                    "paragraphStyle": {
                        "indentFirstLine": {"magnitude": indent_level * 18, "unit": "PT"},
                        "indentStart": {"magnitude": indent_level * 18, "unit": "PT"}
                    },
                    "fields": "indentFirstLine,indentStart"
                }
            })

        # Apply bold styling to @mentions
        mention_offset = 0
        if is_bullet:
            mention_offset = len("  " * indent_level + "• ")
        elif is_checkbox:
            mention_offset = len("☐ ")

        for match in re.finditer(r"@\w+", actual_text):
            mention_start = index + mention_offset + match.start()
            mention_end = index + mention_offset + match.end()
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": mention_start, "endIndex": mention_end},
                    "textStyle": {
                        "bold": True,
                        "foregroundColor": {
                            "color": {
                                "rgbColor": {
                                    "red": 0.2,
                                    "green": 0.4,
                                    "blue": 0.8
                                }
                            }
                        }
                    },
                    "fields": "bold,foregroundColor"
                }
            })

        index += text_length

    return requests

In [4]:
# Example markdown notes
markdown_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# Main execution with error handling
try:
    print("Starting markdown to Google Docs conversion...")

    # Authenticate and create service
    service = authenticate_google_docs()

    # Create new document
    doc_id = create_google_doc(service, "Product Team Sync - May 15, 2023")

    # Parse markdown and generate formatting requests
    print("Parsing markdown content...")
    requests = parse_markdown(markdown_notes)
    print(f"Generated {len(requests)} formatting requests")

    # Update the document with formatting
    print("Applying formatting to document...")
    update_google_doc(service, doc_id, requests)

    print("✅ Conversion completed successfully!")

except Exception as e:
    print(f"❌ Conversion failed: {e}")
    print("Please check your authentication and try again.")

Starting markdown to Google Docs conversion...
Successfully authenticated with Google Docs API
Created document: Product Team Sync - May 15, 2023
Document ID: 1w5ms4ms_zYDc6svwB7LJWpHIsQpG4hxjFEd8K6cpwp8
Document URL: https://docs.google.com/document/d/1w5ms4ms_zYDc6svwB7LJWpHIsQpG4hxjFEd8K6cpwp8
Parsing markdown content...
Generated 77 formatting requests
Applying formatting to document...
Applied batch 1 of 1
Document updated successfully: https://docs.google.com/document/d/1w5ms4ms_zYDc6svwB7LJWpHIsQpG4hxjFEd8K6cpwp8
✅ Conversion completed successfully!
